In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier, BaggingRegressor, AdaBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from time import time
from nltk.corpus import stopwords

In [2]:
import re
import string
import nltk
from nltk.corpus import stopwords

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [4]:
from nltk.stem.snowball import SnowballStemmer

In [5]:
stemmer=SnowballStemmer(language='english')

In [6]:
def stemmizar_texto(texto):
    texto=' '.join([stemmer.stem(palabra) for palabra in texto.split() ])
    return texto

In [7]:
nltk.download('stopwords')
english_stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GyL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [183]:
train_set=pd.read_csv("train.csv",encoding='utf-8')
test_set=pd.read_csv("test.csv",encoding='utf-8')

In [11]:
def quitar_stopwords(texto):
    texto=' '.join([word for word in texto.split() if word not in english_stopwords])
    return texto

def quitar_menciones(texto):
    texto=' '.join([palabra for palabra in texto.split() if '@' not in palabra])
    return texto

In [22]:
Diccionario_de_lenguaje_de_internet={
"2moro": "Tomorrow",
"2nite": "Tonight",
"4EAE": "For Ever And Ever",
"ABT":"About",
"ADN": "Any Day Now",
"AFAIC": "As Far As I’m Concerned",
"AFAICT": "As Far As I Can Tell",
"AFAIK": "As Far As I Know",
"AFAIR": "As Far As I Remember",
"AKA": "Also Known As",
"AMA": "Ask Me Anything",
"ASAIC": "As Soon As I Can",
"ASAP": "As Soon As Possible",
"ATM": "At The Moment",
"B4": "Before",
"B4N": "Bye For Now",
"Bae": "Babe/Before Anyone Else",
"BBL": "Be Back Later",
"BBT": "Be Back Tomorrow",
"BCNU": "Be Seeing You",
"BD": "Big Deal",
"BF": "Boyfriend",
"BFF": "Best Friends Forever",
"BMT": "Before My Time",
"BOL": "Be On Later",
"BOT": "Back On Topic",
"BRB": "Be Right Back",
"BRO": "Brother",
"BT": "But",
"BTW": "By The Way",
"CFY": "Calling For You",
"CU": "See You",
"CUL": "See You Later",
"Cuz": "Because",
"CYA": "Cover Your Ass",
"DAE": "Does Anyone Else",
"DBA": "Doing Business As",
"DFTBA": "Don’t Forget To Be Awesome",
"DIKU": "Do I Know You",
"DM": "Direct Message",
"DND": "Do Not Disturb",
"DR": "Double Rainbow",
"DWBH": "Don’t Worry, Be Happy",
"ELI5": "Explain Like I’m 5",
"EOM": "End Of Message",
"EOS": "End Of Story",
"F2F": "Face To Face",
"FAQ": "Frequently Asked Question",
"FB": "Facebook",
"FBF": "Flash Back Friday",
"FF": "Follow Friday",
"FIFY": "Fixed It For You",
"FITB": "Fill In The Blank",
"FML": "Fuck My Life",
"FOMO": "Fear Of Missing Out",
"FTFY": "Fixed That For You",
"FTL": "For The Loss",
"FTW": "For The Win",
"FWB": "Friends With Benefits",
"FWIW": "For What It’s Worth",
"FYE": "For Your Entertainment",
"FYEO": "For Your Eyes Only",
"FYI": "For Your Information",
"GA": "Go Ahead",
"GAL": "Get A Life",
"GF": "Girlfriend",
"GM": "Good Morning",
"GN": "Good Night",
"Gr8": "Great",
"GTR": "Getting Ready",
"HAND": "Have A Nice Day",
"HB": "Hurry Back",
"HBD": "Happy Birthday",
"HBU": "How About You",
"HMB": "Hit Be Back",
"HMU": "Hit Me Up",
"HRU": "How Are You",
"HTH": "Hope This Helps",
"IAC": "In Any Case",
"IC": "I See",
"ICYMI": "In Case You Missed It",
"IDC": "I Don’t Care",
"IDK": "I Don’t Know",
"IG": "Instagram",
"IIRC": "If I Remember Correctly",
"IKR": "I Know Right",
"ILY": "I Love You",
"IMHO": "In My Humble Opinion",
"IMMD": "It Made My Day",
"IMY": "I Miss You",
"IRL": "In Real Life",
"IS": "I’m Sorry",
"ISO": "In Search Of",
"IU2U": "It’s Up To You",
"J4F": "Just For Fun",
"JAM": "Just A Minute",
"JFY": "Just For You",
"JIC": "Just In Case",
"JK": "Just Kidding",
"JSYK": "Just So You Know",
"KK": "Okay",
"L8": "Late",
"L8R": "Later",
"LMA": "Leave Me Alone",
"LMAO": "Laughing My Ass Off",
"LMBO": "Laughing My Butt Off",
"LMK": "Let Me Know",
"LOL": "Laugh Out Loud",
"LTNS": "Long Time No See",
"LYLAS": "Love You Like A Sister",
"M/F": "Male or Female",
"M8": "Mate",
"MP": "My pleasure",
"MSM": "Mainstream Media",
"MU": "Miss You",
"MYOB": "Mind Your Own Business",
"NAGI": "Not A Good Idea",
"NBD": "No Big Deal",
"NE1": "Anyone",
"NM": "Not Much",
"NP": "No Problem",
"NSFL": "Not Safe For Life",
"NSFW": "Not Safe For Work",
"NTS": "Note To Self",
"NVM": "Never Mind",
"OC": "Original Content",
"OH": "Overheard",
"OIC": "Oh ! I See",
"OMD": "Oh My Damn",
"OMG": "Oh My Goodness",
"OMW": "On My Way",
"OT": "Off Topic",
"OFC": "Of course",
"PAW": "Parents Are Watching",
"Pls": "Please",
"POTD": "Photo Of The Day",
"POV": "Point Of View",
"PPL": "People",
"PTB": "Please Text Back",
"Q4U": "Question For You",
"QQ": "Crying",
"RBTL": "Read Between The Lines",
"RIP": "Rest In Peace",
"RL": "Real Life",
"ROFL": "Rolling On the Floor Laughing",
"RT": "Retweet",
"RTM": "Read The Manual",
"SIS": "Sister",
"SITD": "Still In The Dark",
"SM": "Social Media",
"SMH": "Shaking My Head",
"SMY": "Somebody",
"SNH": "Sarcasm Noted Here",
"SOL": "Sooner Or Later",
"Some1": "Someone",
"SRSLY": "Seriously",
"STBY": "Sucks To Be You",
"Str8": "Straight",
"SYS": "See You Soon",
"TBA": "To Be Announced",
"TBH": "To Be Honest",
"TBT": "Throwback Thursday",
"TBT": "Truth Be Told",
"TFH": "Thread From Hell",
"TFTI": "Thanks For The Invite",
"TGIF": "Thank God It’s Friday",
"THX": "Thanks",
"TIA": "Thanks in Advance",
"TIL": "Today I Learned",
"TIME": "Tears In My Eyes",
"TL;DR": "Too Long; Didn’t Read",
"TLDR":"Too long didn’t read",
"TL DR":"Too long didn’t read",
"TLC": "Tender Loving Care",
"TMI": "Too Much Information",
"TTYL": "Talk To You Later",
"TTYS": "Talk To You Soon",
"Txt": "Text",
"TYVM": "Thank You Very Much",
"U": "You",
"U4F": "You Forever",
"UR": "Your",
"VBG": "Very Big Grin",
"VSF": "Very Sad Face",
"WB": "Welcome Back",
"WBU": "What About You?",
"WEG": "Wicked Evil Grin",
"WKND": "Weekend",
"WOM": "Word of Mouth",
"WOTD": "Word Of The Day",
"Wru": "Who Are You",
"WTH": "What The Heck?",
"WTPA": "Where The Party At?",
"WU?": "What's Up",
"WU":"What's Up",
"WYCM": "Will You Call Me?",
"WYWH": "Wish You Were Here",
"XOXO": "Hugs and Kisses",
"YGM": "You’ve Got Mail",
"YNK": "You Never Know",
"YOLO": "You Only Live Once",
"YT": "YouTube",
"YW": "You’re Welcome",
}

In [13]:
def reemplazar_lenguaje_internet(texto):
    texto=texto.upper()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=Diccionario_de_lenguaje_de_internet.get(palabra,'not internet slang')
        if(traduccion!='not internet slang'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [136]:
train_set

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


### Feature: Cantidad de caracteres usados en tweets

In [184]:
cuenta_caract = train_set[['id','text']]
cuenta_caract['caracteres_usados'] = cuenta_caract['text'].str.len()
cuenta_caract_t = test_set[['id','text']]
cuenta_caract_t['caracteres_usados'] = cuenta_caract_t['text'].str.len()

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [185]:
cuenta_caract = cuenta_caract[['id','caracteres_usados']]
cuenta_caract_t = cuenta_caract_t[['id','caracteres_usados']]

In [186]:
train_set=train_set.merge(cuenta_caract,on='id',how='left')

In [187]:
test_set=test_set.merge(cuenta_caract_t,on='id',how='left')

### Feature: cantidad de menciones por tweet

In [188]:
cant_menciones = train_set[['id','text']]
cant_menciones['menciones_realizadas']=cant_menciones['text'].str.count('@')
cant_menciones_t = test_set[['id','text']]
cant_menciones_t['menciones_realizadas']=cant_menciones_t['text'].str.count('@')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [189]:
cant_menciones = cant_menciones[['id','menciones_realizadas']]
cant_menciones_t=cant_menciones_t[['id','menciones_realizadas']]

In [190]:
train_set=train_set.merge(cant_menciones,on='id',how='left')

In [191]:
test_set=test_set.merge(cant_menciones_t,on='id',how='left')

### Feature: ID que comparten localización

In [192]:
tiene_localizacion = train_set[['id','location']]
tiene_localizacion_t = test_set[['id','location']]

In [193]:
tiene_localizacion['location'].loc[~(tiene_localizacion['location'].isnull())]=1
tiene_localizacion_t['location'].loc[~(tiene_localizacion_t['location'].isnull())]=1

C:\Users\GyL\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [194]:
tiene_localizacion.fillna(0,inplace=True)
tiene_localizacion_t.fillna(0,inplace=True)

C:\Users\GyL\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [195]:
tiene_localizacion['location'].value_counts()

1    5080
0    2533
Name: location, dtype: int64

In [196]:
tiene_localizacion = tiene_localizacion.rename(columns={'location':'permite_location'})
tiene_localizacion_t = tiene_localizacion_t.rename(columns={'location':'permite_location'})

In [197]:
train_set=train_set.merge(tiene_localizacion,on='id',how='left')

In [198]:
test_set=test_set.merge(tiene_localizacion_t,on='id',how='left')

### Feature: usa Keyword

In [199]:
usa_keyword = train_set[['id','keyword']]
usa_keyword['keyword'].loc[~(usa_keyword['keyword'].isnull())] = 1
usa_keyword_t = test_set[['id','keyword']]
usa_keyword_t['keyword'].loc[~(usa_keyword_t['keyword'].isnull())] = 1

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [200]:
usa_keyword.fillna(0,inplace=True)
usa_keyword_t.fillna(0,inplace=True)

In [201]:
usa_keyword = usa_keyword.rename(columns={'keyword':'use_keyword'})
usa_keyword_t = usa_keyword_t.rename(columns={'keyword':'use_keyword'})

In [202]:
train_set = train_set.merge(usa_keyword,on='id',how='left')

In [203]:
test_set=test_set.merge(usa_keyword_t,on='id',how='left')

### Feature: Cita URL en Tweet

In [204]:
usa_url = train_set[['id','text']]
usa_url_t = test_set[['id','text']]

In [205]:
usa_url['cita_url'] = usa_url['text'].str.count('http')
usa_url_t['cita_url'] = usa_url_t['text'].str.count('http')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [206]:
usa_url = usa_url[['id','cita_url']]
usa_url_t = usa_url_t[['id','cita_url']]

In [207]:
train_set = train_set.merge(usa_url,on='id',how='left')

In [208]:
test_set=test_set.merge(usa_url_t,on='id',how='left')

### Feature: usa Hashtag

In [209]:
usa_hashtag = train_set[['id','text']]
usa_hashtag['use_hashtag']=usa_hashtag['text'].str.count('#')
usa_hashtag_t = test_set[['id','text']]
usa_hashtag_t['use_hashtag']=usa_hashtag_t['text'].str.count('#')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [210]:
usa_hashtag = usa_hashtag[['id','use_hashtag']]
usa_hashtag_t = usa_hashtag_t[['id','use_hashtag']]

In [211]:
train_set = train_set.merge(usa_hashtag,on='id',how='left')

In [212]:
test_set=test_set.merge(usa_hashtag_t,on='id',how='left')

In [215]:
train_set.to_csv('train_data.csv')

In [216]:
test_set.to_csv('test_data.csv')

In [213]:
train_set

,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0,0,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0,0,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,83,0,0,0,1,0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,125,2,0,0,0,0
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,0,0,0,1,0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,137,0,0,0,0,0


In [214]:
test_set

,id,keyword,location,text,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag
0,0,NaN,NaN,Just happened a terrible car crash,34,0,0,0,0,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,0,0,0,0,1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,0,0,0,0,0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,0,0,0,0,2
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,55,0,0,0,0,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,139,0,0,0,0,0
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,55,0,0,0,1,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,65,0,0,0,1,0


In [297]:
prueba['hola'] = prueba['text'].apply(reemplazar_lenguaje_internet)